# Process visualization data

This notebook processes the CovidCareMap US Healthcare System Capacity data into the format used with the `viz/heathcare-system-capacity` visualization.

It also move the HGHI and ventilator into the `viz/hghi-vents` app.

In [ ]:
import shutil
import os
import json

import numpy as np

from covidcaremap.data import (PUBLISHED_DATA_DIR, 
                               DATA_DIR, 
                               PROCESSED_DATA_DIR, 
                               processed_data_path)

### Data for US Health System Capacity app

In [ ]:
viz_data_dir = os.path.join(DATA_DIR, '../viz/us-healthcare-system-capacity/data')

viz_input_dir = os.path.join(viz_data_dir, 'input')
viz_config_dir = os.path.join(viz_data_dir, 'config')

!mkdir -p $viz_input_dir
!mkdir -p $viz_config_dir

data_paths = [
    (os.path.join(PUBLISHED_DATA_DIRv, x), os.path.join(viz_input_dir, x))
    for x in [
        'us_healthcare_capacity-county-CovidCareMap.geojson',
        'us_healthcare_capacity-facility-CovidCareMap.geojson',
        'us_healthcare_capacity-hrr-CovidCareMap.geojson',
        'us_healthcare_capacity-state-CovidCareMap.geojson'
    ]
] + [
    (os.path.join(PROCESSED_DATA_DIR, x), os.path.join(viz_config_dir, x))
    for x in [
        'ccm_county_breaks.json',
        'ccm_state_breaks.json',
        'ccm_hrr_breaks.json',
        'ccm_facility_breaks.json'
    ]
]

In [ ]:
for src, dst in data_paths:
    print('Copying {} to {}'.format(src, dst))
    # Tippecanoe doesn't read NaN values, so convert to nulls
    if src.endswith('.geojson'):
        with open(src) as f:
            gj = json.loads(f.read())
        for feat in gj['features']:
            for prop in feat['properties']:
                v = feat['properties'][prop]
                if type(v) is float and np.isnan(v):
                    feat['properties'][prop] = None
        with open(dst, 'w') as f:
            f.write(json.dumps(gj, indent=2))
    else:
        shutil.copy(src, dst)

In [ ]:
!cd $viz_data_dir && ./process.sh

### Data for Ventilator Supply and Healthcare Capacity Map, by State  app

In [ ]:
hghi_vents_data_dir = os.path.join(DATA_DIR, '../viz/hghi-vents/public')

In [ ]:
hghi_vents_data = processed_data_path('hghi_state_data_with_vents.geojson')

In [ ]:
!cp $hghi_vents_data $hghi_vents_data_dir